In [ ]:
!pip install text2graphapi

In [ ]:
import pandas as pd
import networkx as nx
import numpy as np
from networkx.algorithms import approximation

In [ ]:
datos = pd.read_json('subtask_1.jsonl', lines=True)

In [ ]:
data_recortada = datos.head(54832)

In [ ]:
data_recortada.drop('id', axis=1, inplace=True)

In [ ]:
data_recortada['filas'] = range(1, len(data_recortada) + 1)

In [ ]:
data_recortada

In [ ]:
text_to_graph = []
for index, row in data_recortada.iterrows():
    fila = {"id": row['filas'], "doc": row['text']}
    text_to_graph.append(fila)

In [ ]:
from text2graphapi.src.Cooccurrence import Cooccurrence

to_word_coocc_graph = Cooccurrence(graph_type = 'DiGraph',
        language = 'es', apply_prep = False,
        window_size = 3, output_format = 'networkx')

In [ ]:
listilla = []
block_size = 1000


for i in range(0, len(text_to_graph), block_size):

    block = text_to_graph[i:i + block_size]


    grafos_cooncurrencia = to_word_coocc_graph.transform(block)


    listilla.append(grafos_cooncurrencia)

In [ ]:
lista_de_verdad = []

for j in range(len(listilla[0:42])):
  for i in listilla[j]:
    grafo = i['graph']
    lista_de_verdad.append(grafo)

In [ ]:
def get_avg_degree_centr(graph: nx.Graph):     #este sí me sirve
    centrality = list(nx.degree_centrality(graph).values())
    count = np.mean(centrality)
    return count

In [ ]:
grado_centralidad = []      #sí me sirve esta metrica
for i in lista_de_verdad:
  grado_centralidad.append(get_avg_degree_centr(i))


In [ ]:
def get_clustering_approx(graph: nx.DiGraph):      #este sí me sirve
    graph_und = graph.to_undirected()
    clust = approximation.average_clustering(graph_und)
    return clust


In [ ]:
clust = []                 #este sí me sirve
for grafo in lista_de_verdad:
  clust.append(get_clustering_approx(i))


len(clust)

In [ ]:
def get_number_components(graph: nx.Graph):       #no se si me sirve
    g = graph.to_undirected()
    num_components = nx.number_connected_components(g)/g.number_of_nodes()

    return num_components

In [ ]:
components = []           #no se si me srive

for grafo in lista_de_verdad:
  components.append(get_number_components(grafo))



In [ ]:
def get_number_strong_comp(graph: nx.Graph):   #no sé si me sirve x2
    num_strong_components = nx.number_strongly_connected_components(graph)/graph.number_of_nodes()
    return num_strong_components


In [ ]:
strongest = []      #no sé si me sirve x2
for grafo in lista_de_verdad:
  strongest.append(get_number_strong_comp(grafo))



In [ ]:

def entropy(cent_measure):
    m = np.asarray(cent_measure, float)
    dist = m / m.sum()
    ent = np.nansum(dist * np.log2(1.0 / dist))
    return ent

In [ ]:
def get_entropy_clos(graph: nx.Graph):
    centrality = list(nx.closeness_centrality((graph)).values())
    num_entropy = entropy(centrality)/graph.number_of_nodes()
    return num_entropy


In [ ]:
entropy_clos = []
for grafo in lista_de_verdad:
  entropy_clos.append(get_entropy_clos(grafo))


In [ ]:
def get_transitivity(graph: nx.DiGraph):
    trans = nx.transitivity(graph)
    return trans


In [ ]:
trans = []
for grafo in lista_de_verdad:
  trans.append(get_transitivity(grafo))

In [ ]:
def get_sparseness(graph: nx.Graph):
    mat = nx.adjacency_matrix((graph)).todense()
    num_num = np.count_nonzero(mat)
    num_val = np.prod(mat.shape)
    sparseness = float(num_num) / num_val
    return sparseness

In [ ]:
sparse = []
for grafo in lista_de_verdad:
  sparse.append(get_sparseness(grafo))

In [ ]:
def get_entropy_centr(graph: nx.Graph):
    centrality = list(nx.degree_centrality((graph)).values())
    num_entropy = entropy(centrality)/graph.number_of_nodes()
    return num_entropy


In [ ]:
entropy_centr = []

for grafo in lista_de_verdad:
  entropy_centr.append(get_entropy_centr(grafo))

In [ ]:
def get_degree_connectivity(graph: nx.Graph):
    count = np.mean(list(nx.average_neighbor_degree(graph).values()))
    return count/graph.number_of_nodes()


In [ ]:
neighbor_grado_1 = []
for i in lista_de_verdad:
  neighbor_grado_1.append(get_degree_connectivity(i))

In [ ]:
def get_assortativity(graph: nx.Graph):
    try:
        assort = nx.degree_pearson_correlation_coefficient(graph)
    except:
        assort = np.nan
    return abs(assort)

In [ ]:
asortaty = []
for i in lista_de_verdad:
  asortaty.append(get_assortativity(i))

In [ ]:
data_final = pd.read_csv('metricas_subtask_2.csv')

In [ ]:
data_final

In [ ]:
nueva_fila = {'cercania':0.358906, 'eficiencia':0.449429, 'clustering':0.327558}

data_final.loc[len(data_final)] = nueva_fila

In [ ]:
data_final = data_final.rename(columns={'cercania': 'clossenes',
                                        'eficiencia':'global_efficiency',
                                        'clustering':'clustering'})

In [ ]:
datos_otravez = pd.read_json('subtask_2.jsonl', lines=True)

In [ ]:
recortados = datos_otravez.head(28000)


In [ ]:
len(recortados['id']), len(recortados['label']), len(data_final['clustering']), len(data_final['global_efficiency']),len(data_final['clossenes']), len(grado_centralidad)

(29378, 29378, 29378, 29378, 29378, 29378)

In [ ]:
datos_YA = pd.DataFrame({'id':recortados['id'],
                         'label': recortados['label'],
                         'clustering':data_final['clustering'],
                         'global_efficiency':data_final['global_efficiency'],
                         'clossenes':data_final['clossenes'],
                         'degree_centrality':grado_centralidad,
                         'neighbor_degree':neighbor_grado_1,
                         'assortativity':asortaty,
                         'transitivity':trans,
                         'number_strong_comp':strongest,
                         'entropy_centr': entropy_centr,
                         'entropy_clos':entropy_clos,
                         'sparseness':sparse})


In [ ]:
datos_YA.head(3)

In [ ]:
datos_YA.to_csv('metricas_calculadas_subtask_2.csv', index=False)

In [ ]:
datos_YA.info()